# Introducing Kotlin **Channels**

- Kotlin **Channels** are a powerful feature of Kotlin **Coroutines** that provide a way to **communicate between coroutines**. They are similar to blocking queues and allow for asynchronous data exchange while providing safe synchronization.
- **Non-blocking**: Channels enable asynchronous communication without blocking threads, making it easier to build responsive applications.
- **Channels** facilitate communication between coroutines by allowing one coroutine to send data and another to receive it.
- **Hot stream**: Unlike flows, channels are **hot streams**, meaming they continuously produce values and can emit data regardless of whether there are active collectors.
- Types of Channels:
  1. **Rendezvous Channels**: These are the default channels where the sender is suspended until the receiver is ready to receive the data.
  2. **Buffered Channels**: These channels *can hold a specified number of elements*, allowing the sender to send data *without waiting for the receiver* until the buffer is full.
  3. **Conflated Channels**: These *only keep the latest value sent to the channel*, discarding older values. This is useful for scenarios where only the latest data matters.



In [1]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext
import kotlinx.coroutines.flow.*
import kotlin.system.measureTimeMillis
import kotlinx.coroutines.channels.*

## Creating Rendezvous Channels

- **Rendezvous** channels, *suspend* when they `send` a value until it is received.

In [2]:
// Creating a simple rendezvouse channel

fun main() = runBlocking {

    /*
    This channel has no buffer. It is a rendezvous channel.
    So it emits new data when the previous one is collected by a receiver.
    */
    val channel = Channel<Int>() // Creating a channel that carries Int data

    // Launching a coroutine to send values
    launch {
        repeat(5) { x ->
            delay(500)
            println("Sent value: $x")
            channel.send(x) // Sending values to the channel. Suspends here until a receiver collects the value. 
        }
        
        /* 
        It means that no more elements will/can be sent through the channel.
        But receiver can still recieve all data already sent through channel.
        */
        channel.close() // Closing the channel
    }

    // Launching a coroutine to receive values
    launch {
        
        for (x in 1 until 6) { // Receives the channel data
            delay(1000)
            val v = channel.receive() 
            println("Received value: $v") // Print received values
        }

        // Or alternatively we can use 'receive'
        /*
        for (v in channel) { // Receives the channel data
            delay(1000)
            println("received value: $v") // Print received values
        }
        */
    }
}

main()

Sent value: 0
Received value: 0
Sent value: 1
Received value: 1
Sent value: 2
Received value: 2
Sent value: 3
Received value: 3
Sent value: 4
Received value: 4


StandaloneCoroutine{Completed}@3248e063

## Creating Buffered Channels

- A **buffered** channel can be created by `Channel<Int>(SIZE)`.
- **Buffered** channels, *suspend* when they `send` a value only **IF** the buffer is full.
- Buffers are first-in-first-out (**FIFO**).

In [3]:
// Creating a simple buffered channel

fun main() = runBlocking {

    /*
    This channel has a buffer with size 3.
    The buffer allows 3 values to be sent without suspending.
    The producer will only suspend when the buffer is full (i.e., 3 elements).
    */
    // IMPORTANT: Buffer is first-in-first-out (FIFO)
    val channel = Channel<Int>(3) // Creating a channel that carries Int data

    // Launching a coroutine to send values
    launch {
        repeat(10) { x ->
            delay(500)
            println("Sent value: $x")
            channel.send(x) // Suspends here if buffer is full (after sending 3 values).
        }
        
        /* 
        It means that no more elements will/can be sent through the channel.
        But receiver can recieve all data already sent through channel.
        */
        channel.close() // Closing the channel
    }

    // Launching a coroutine to receive values
    launch {
        // The receiver starts after a 5-second delay, giving the producer enough time to fill the buffer.
        delay(5_000)
        println("Receiving values started.")
        for (v in channel) { // Receives the channel data
            delay(1000)
            println("Received value: $v") // Print received values
        }
    }
}

main()

Sent value: 0
Sent value: 1
Sent value: 2
Sent value: 3
Receiving values started.
Sent value: 4
Received value: 0
Sent value: 5
Received value: 1
Sent value: 6
Received value: 2
Sent value: 7
Received value: 3
Sent value: 8
Received value: 4
Sent value: 9
Received value: 5
Received value: 6
Received value: 7
Received value: 8
Received value: 9


StandaloneCoroutine{Completed}@2e0ac167

## Creating Conflated Channels

- A **conflated** channel can be created by `Channel<Int>(Channel.CONFLATED)`.
- In a **conflated** channel, a slow receiver might lose data because the channel *only holds latest* data.
- In a **conflated** channel, `send` **never suspends**. The channel only holds the most recent value, discarding any previous ones if new data is sent before the receiver collects it.


In [4]:
// Creating a simple conflated channel

fun main() = runBlocking {

    /*
    This channel is conflated, meaning it will only keep the most recent value.
    If the receiver is slow, older values will be dropped, and the latest will be sent.
    */
    val channel = Channel<Int>(Channel.CONFLATED) // Creating a conflated channel

    // Launching a coroutine to send values
    launch {
        repeat(10) { x ->
            delay(1000)
            println("Sent value: $x")
            channel.send(x) // Conflated channel will only keep the most recent value.
        }
        
        channel.close()
    }

    // Launching a coroutine to receive values
    launch {
        // The receiver starts after a 5-second delay, meaning the earlier values will be lost.
        delay(5_000)
        println("Receiving values started.")
        for (v in channel) { // Receives the most recent value in the conflated channel.
            delay(1000)
            println("Received value: $v") // Prints the latest value from the conflated channel.
        }
    }
}

main() // In conflated channel, a slow receiver might lose data because the channel only holds latest data.


Sent value: 0
Sent value: 1
Sent value: 2
Sent value: 3
Receiving values started.
Sent value: 4
Received value: 3
Sent value: 5
Received value: 4
Sent value: 6
Received value: 5
Sent value: 7
Received value: 6
Sent value: 8
Received value: 7
Sent value: 9
Received value: 8
Received value: 9


StandaloneCoroutine{Completed}@9c02129

## Receiving Channel Data from Multiple Coroutines

- Unlike flows, channels can have **multiple cuncurrent receivers**. This is called **Fan-out**.
- Channels can be **shared** between **multiple consumers**. Multiple receivers can read from the same channel concurrently.
- Collectors **compete to collect** data from channel; faster ones get more data.

In [5]:
// Receiving channel data from two different coroutines

fun main() = runBlocking {

    val channel = Channel<Int>(3) 

    // Launching a coroutine to send values
    launch {
        repeat(10) { x ->
            println("Sent value: $x")
            channel.send(x) 
        }
        channel.close() // Closing the channel
    }

    /*
    The first reciever that sends a receive request, gets the data.
    */
    
    // Launching the first coroutine to receive values
    launch {
        for (v in channel) { 
            delay(1000)
            println("Received value in corotuine 1: $v") // Print received values
        }
    }

    // Launching the second coroutine to receive values
    launch {
        for (v in channel) { // Receives the channel data
            delay(500)
            println("Received value in corotuine 2: $v") // Print received values
        }
    }
}

main()

Sent value: 0
Sent value: 1
Sent value: 2
Sent value: 3
Sent value: 4
Sent value: 5
Received value in corotuine 2: 1
Sent value: 6
Received value in corotuine 1: 0
Received value in corotuine 2: 2
Sent value: 7
Sent value: 8
Received value in corotuine 2: 4
Sent value: 9
Received value in corotuine 1: 3
Received value in corotuine 2: 5
Received value in corotuine 2: 7
Received value in corotuine 1: 6
Received value in corotuine 2: 8
Received value in corotuine 1: 9


StandaloneCoroutine{Completed}@206cfe94

## Sending Data through Channel from Multiple Coroutines

- Multiple coroutines may send to the same channel. This is called **Fan-in**.
- Channels can be used for **communication** between coroutines, where **one or more** producers send data to **one or more** consumers.

In [6]:
// Sending data through the channel from two coroutines

fun main() = runBlocking {

    val channel = Channel<Int>(3) 

    launch {
        coroutineScope { // Closing the channel after 2 seconds
            // Launching the first coroutine to send values
            launch {
                for (x in 1 until 6) {
                    delay(100)
                    println("Sent value from coroutine 1: $x")
                    channel.send(x) 
                }
            }

            // Launching the second coroutine to send values
            launch {
                for (x in 6 until 11) {
                    delay(200)
                    println("Sent value from coroutine 2: $x")
                    channel.send(x) 
                }
            }
        }
        channel.cancel()
    }
    
    
    // Launching a coroutine to receive values
    launch {
        for (v in channel) { 
            delay(1000)
            println("Received value: $v") // Print received values
        }
    }
}

main()

Sent value from coroutine 1: 1
Sent value from coroutine 2: 6
Sent value from coroutine 1: 2
Sent value from coroutine 1: 3
Sent value from coroutine 2: 7
Sent value from coroutine 1: 4
Received value: 1
Sent value from coroutine 2: 8
Received value: 6
Sent value from coroutine 1: 5
Received value: 2
Sent value from coroutine 2: 9
Received value: 3
Received value: 7
Sent value from coroutine 2: 10
Received value: 4
Received value: 8


StandaloneCoroutine{Cancelled}@616ce725

## Channel Producers and `ReceiveChannel`

- We can use `produce` **corotuine builder** to create and return a specific type of channel, e.g. `ReceiveChannel`.
- `ReceiveChannel` is a channel that can only be used to receive data; we cannot send data to it anywhere else outside of the `produce` corotuine builder.
- When using `produce` **corotuine builder**, the resulting channel has a default buffer size 16.

In [7]:
// Using 'produce' cortoutine builder to create a 'ReceiveChannel'
@OptIn(kotlinx.coroutines.ExperimentalCoroutinesApi::class)

/*
The coroutine builder 'produce' creates a 'ReceiveChannel'.
We can set the buffer size by 'capacity = bufferSize'

IMPORTANT: To define 'produce' outside of a coroutine scope, 
it needs to be defined as an extension function of `CoroutineScope` 
to receive the coroutine context of its caller. 
*/

fun CoroutineScope.myProducer(): ReceiveChannel<Int> = produce(capacity = 3) {
    for (x in 1 until 10) { 
        delay(250)
        println("Produced value: $x")
        send(x)
    }
    
    // Explicitly close the channel (optional, as it will close automatically)
    // channel.close()
}


fun main() = runBlocking {

    val channel = myProducer() // Creating a receiver channel
    
    // Launching a coroutine to receive values
    launch {
        for (v in channel) { 
            delay(1000)
            println("Received value: $v") // Print received values
        }
    }

}

main()

Produced value: 1
Produced value: 2
Produced value: 3
Produced value: 4
Received value: 1
Produced value: 5
Produced value: 6
Received value: 2
Produced value: 7
Received value: 3
Produced value: 8
Received value: 4
Produced value: 9
Received value: 5
Received value: 6
Received value: 7
Received value: 8
Received value: 9


StandaloneCoroutine{Completed}@4ff4aed5

## Piplines

- We can use `produce` and `ReceiveChannel` to create a pipeline of channels and data

In [8]:
// Creating a pipeline
@OptIn(kotlinx.coroutines.ExperimentalCoroutinesApi::class)

// The first channel creates numbers
fun CoroutineScope.firstProducer(): ReceiveChannel<Int> = produce {
    for (x in 1 until 10) { 
        delay(250)
        send(x)
    }
}

// The second channel creates a string using the data received from the first channel
fun CoroutineScope.secondProducer(channel: ReceiveChannel<Int>): ReceiveChannel<String> = produce {
    for (x in channel) { 
        delay(100)
        send("value is $x")
    }
}

fun main() = runBlocking {

    val baseChannel = firstProducer() // Creating base producer
    val channel = secondProducer(baseChannel) // Creating a receiver channel
    
    // Launching a coroutine to receive values
    launch {
        for (v in channel) { 
            delay(1000)
            println("Received $v") // Print received values
        }
    }

}

main()

Received value is 1
Received value is 2
Received value is 3
Received value is 4
Received value is 5
Received value is 6
Received value is 7
Received value is 8
Received value is 9


StandaloneCoroutine{Completed}@39c761e2

## **DYE**: Do Your Experiment

In [9]:
// Ready for your codes